# Importamos librerias

In [1]:
#pip install snowflake-connector-python==2.7.6

In [2]:
import pandas as pd
import numpy as np
import snowflake.connector
import json
import os
from snowflake.connector.pandas_tools import write_pandas
from datetime import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta
pd.options.mode.chained_assignment = None

In [3]:
while True:
    try:

        f = open('credentials.json')
        data_pass = json.load(f)

        pass_ = input("INGRESAR PASSCODE:")

        ctx = snowflake.connector.connect(
            user = data_pass['snow']['USER'],
            password = data_pass['snow']['PASS'],
            account = data_pass['snow']['ACCOUNT'],
            passcode = pass_,
            database = 'SANDBOX_PLUS',
            schema = 'DWH'
        )

        cursor = ctx.cursor()

        print('Connected')

        break

    except:
        print('Incorrect Password - provide again')

    print('Correct Password - connected to SNOWFLAKE')

Connected


# Segmentos para Redes

### Notas

In [4]:
# sumar una hoja al excel que contenga
#
# subclase
# clientes activos tata (clientes que compraron en tata ultimos 3 meses)
# clientes activos tata que compran la subclase
# clientes activos tata que no compran la subclase y tienen alto score (scoring de bzm)
# clientes activos tata que no compran la subclase y tienen alto score (scoring de bzm) | contactables mail
# clientes activos tata que no compran la subclase y tienen alto score (scoring de bzm) | contactables celu
#
# (entender con fede si la contactabilidad la queremos excluyente o no. mencionó algo de enviar x wpp, por lo que capaz solo importa contactabilidad celular)

In [5]:
# 1. clientes que compran el articulo y quiero venderles mas
# 2. clientes que no compran el articulo pero si compran articulos de la subclase
# 2.2. idem punto 2 pero para score 4 o 5 en la subclase
# 3. clientes que no compran y tienen score 5, y hayan comprado en TATA en los ultimos 3 meses
# 4. clientes que compran la subclase, pero no compran la marca
# 5. clientes que en los ultimos 30 dis NO VENGAN A TATA y que tengan score 5 en esa clase
# que sea para whatsapp
#
# enviar segmento como sheets adicionales
# periodo de tiempo → un mes

### Articulos destacados

In [6]:
url_fede = 'https://docs.google.com/spreadsheets/d/1JQ23ubzz9W6CteBHoHdqZtl9dp5zmSfTQf0F9Du29R8/export?format=csv'
articulos_destacados = pd.read_csv(url_fede)
articulos_destacados.head(2)

,Categoria,Estadistico,ORIN,Desc,Cant Clientes
0,Alfajores,260668000,1000054862,ALFAJOR NIEVE PORTEZUELO PACK 6.00 U,NaN
1,Alfajores,260669000,1000054863,ALFAJOR CHOCOLATE PORTEZUELO PACK 6.00 U,NaN


In [7]:
query = '''
SELECT
    SUB.SUB_NAME,
    SUB.SUBCLASE,
    LAB.ARTC_BRAND_DESC AS MARCA,
    LAA.ORIN,
    LAA.ARTC_ARTC_DESC AS ARTICULO
FROM
    MSTRDB.DWH.LU_ARTC_BRAND AS LAB
    INNER JOIN MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA ON LAA.ARTC_BRAND_ID = LAB.ARTC_BRAND_ID
    INNER JOIN MSTRDB.DWH.ITEM_MASTER AS IM ON IM.ITEM = LAA.ORIN
    INNER JOIN MSTRDB.DWH.SUBCLASS AS SUB ON SUB.SUBCLASE = IM.SUBCLASE
WHERE
    LAA.ORIN IN {orin_snow}
'''

cursor.execute(query.format(orin_snow = tuple(str(number) for number in articulos_destacados['ORIN'])))
articulos_destacados_2 = cursor.fetch_pandas_all()
print(articulos_destacados.shape[0] == articulos_destacados_2.shape[0])
articulos_destacados_2.head(2)

True


,SUB_NAME,SUBCLASE,MARCA,ORIN,ARTICULO
0,HELADOS 2 LITROS,3200010002,CRUFI,1000099233,HELADO CRUFI 2.00 L SAMBAYON/CHOCOLATE/D
1,HELADOS 2 LITROS,3200010002,CRUFI,1000099234,HELADO CRUFI 2.00 L CREMA VETEADA


In [8]:
print(f"ORINES --> {articulos_destacados_2['ORIN'].nunique()}")
print(f"MARCAS --> {articulos_destacados_2['MARCA'].nunique()}")
print(f"SUBCLASES --> {articulos_destacados_2['SUB_NAME'].nunique()}")

ORINES --> 27
MARCAS --> 12
SUBCLASES --> 13


### Segmentos

#### 1. Clientes que ya compran el articulo

In [9]:
query = '''
SELECT
    DISTINCT
    SUB.SUB_NAME,
    SUB.SUBCLASE,
    LAB.ARTC_BRAND_DESC AS MARCA,
    LAA.ORIN,
    LAA.ARTC_ARTC_DESC AS ARTICULO,
    CONT.SOCI_SOCI_ID,
    CONT.CLIE_TIPO_DOC,
    CONT.CLIE_CLIE_NDOC,
    CASE
        WHEN CONT.CLIE_TIPO_DOC LIKE 'C' THEN 'CI:' || RIGHT(CONT.CLIE_CLIE_NDOC, 4)
        ELSE 'Doc.' || RIGHT(CONT.CLIE_CLIE_NDOC, 4)
        END AS ULTIMOS_4,
    CONT.CELULAR
FROM
    MSTRDB.DWH.LU_ARTC_BRAND AS LAB
    INNER JOIN MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA ON LAA.ARTC_BRAND_ID = LAB.ARTC_BRAND_ID
    INNER JOIN MSTRDB.DWH.ITEM_MASTER AS IM ON IM.ITEM = LAA.ORIN
    INNER JOIN MSTRDB.DWH.SUBCLASS AS SUB ON SUB.SUBCLASE = IM.SUBCLASE
    INNER JOIN MSTRDB.DWH.FT_VENTAS AS FV ON FV.ARTC_ARTC_ID = LAA.ARTC_ARTC_ID
    INNER JOIN MSTRDB.DWH.FT_FDLN_MOVIMIENTOS AS FFM ON FFM.TICKET = FV.TICKET
    INNER JOIN SANDBOX_PLUS.DWH.CONTACTABLES_2 AS CONT ON CONT.SOCI_SOCI_ID = FFM.SOCI_SOCI_ID
    INNER JOIN MSTRDB.DWH.LU_CLIE_CLIENTE AS LCC ON LCC.SOCI_SOCI_ID = FFM.SOCI_SOCI_ID
WHERE
    FFM.TIEM_DIA_ID >= DATEADD(MONTH, -1, CURRENT_DATE)
    AND CONT.CELULAR IS NOT NULL
    AND CONT.WP = 1
    AND CONT.SOCI_SOCI_ID NOT IN (SELECT SOCI_SOCI_ID FROM SANDBOX_PLUS.DWH.SOCIOS_NO_COMUNICAR)
    AND ( LCC.CLIE_CLIE_NOMBRE1 NOT IN('No Disponible', 'No disponible', 'N A') OR (LCC.CLIE_CLIE_NOMBRE1 = 'No Disponible' AND LCC.CLIE_CLIE_FNAC = '1919-01-01') )
    AND LAA.ORIN IN {orin_snow}
'''

cursor.execute(query.format(orin_snow = tuple(str(number) for number in articulos_destacados_2['ORIN'])))
segm_1 = cursor.fetch_pandas_all()
print(segm_1.shape[0])
segm_1.head(2)

92016


,SUB_NAME,SUBCLASE,MARCA,ORIN,ARTICULO,SOCI_SOCI_ID,CLIE_TIPO_DOC,CLIE_CLIE_NDOC,ULTIMOS_4,CELULAR
0,CHOCOLATADA,7600030001,PARMALAT,1000408185,LECHE CHOCOLATADA PARMALAT LARGA DURACION SACH...,1113664,C,53509441,CI:9441,97588465
1,CHOCOLATADA,7600030001,PARMALAT,1000408185,LECHE CHOCOLATADA PARMALAT LARGA DURACION SACH...,1327920,C,41998624,CI:8624,98279377


In [10]:
segm_1[segm_1.duplicated(subset = ['SOCI_SOCI_ID', 'ORIN'], keep = False)]

,SUB_NAME,SUBCLASE,MARCA,ORIN,ARTICULO,SOCI_SOCI_ID,CLIE_TIPO_DOC,CLIE_CLIE_NDOC,ULTIMOS_4,CELULAR


#### 2.1. Clientes que compran la subclase pero no el articulo

In [11]:
query = '''
WITH COMPRARON AS
    (
    SELECT
        DISTINCT
        FFM.SOCI_SOCI_ID
    FROM
        MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA
        INNER JOIN MSTRDB.DWH.FT_VENTAS AS FV ON FV.ARTC_ARTC_ID = LAA.ARTC_ARTC_ID
        INNER JOIN MSTRDB.DWH.FT_FDLN_MOVIMIENTOS AS FFM ON FFM.TICKET = FV.TICKET
    WHERE
        FFM.TIEM_DIA_ID >= DATEADD(MONTH, -1, CURRENT_DATE)
        AND LAA.ORIN = '{orin_snow}'
    )

SELECT
    DISTINCT
    SUB.SUB_NAME,
    SUB.SUBCLASE,
    CONT.SOCI_SOCI_ID,
    CONT.CLIE_TIPO_DOC,
    CONT.CLIE_CLIE_NDOC,
    CASE
        WHEN CONT.CLIE_TIPO_DOC LIKE 'C' THEN 'CI:' || RIGHT(CONT.CLIE_CLIE_NDOC, 4)
        ELSE 'Doc.' || RIGHT(CONT.CLIE_CLIE_NDOC, 4)
        END AS ULTIMOS_4,
    CONT.CELULAR
FROM
    MSTRDB.DWH.LU_ARTC_BRAND AS LAB
    INNER JOIN MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA ON LAA.ARTC_BRAND_ID = LAB.ARTC_BRAND_ID
    INNER JOIN MSTRDB.DWH.ITEM_MASTER AS IM ON IM.ITEM = LAA.ORIN
    INNER JOIN MSTRDB.DWH.SUBCLASS AS SUB ON SUB.SUBCLASE = IM.SUBCLASE
    INNER JOIN MSTRDB.DWH.FT_VENTAS AS FV ON FV.ARTC_ARTC_ID = LAA.ARTC_ARTC_ID
    INNER JOIN MSTRDB.DWH.FT_FDLN_MOVIMIENTOS AS FFM ON FFM.TICKET = FV.TICKET
    INNER JOIN SANDBOX_PLUS.DWH.CONTACTABLES_2 AS CONT ON CONT.SOCI_SOCI_ID = FFM.SOCI_SOCI_ID
    INNER JOIN MSTRDB.DWH.LU_CLIE_CLIENTE AS LCC ON LCC.SOCI_SOCI_ID = FFM.SOCI_SOCI_ID
WHERE
    FFM.TIEM_DIA_ID >= DATEADD(MONTH, -1, CURRENT_DATE)
    AND CONT.CELULAR IS NOT NULL
    AND CONT.WP = 1
    AND CONT.SOCI_SOCI_ID NOT IN (SELECT SOCI_SOCI_ID FROM SANDBOX_PLUS.DWH.SOCIOS_NO_COMUNICAR)
    AND ( LCC.CLIE_CLIE_NOMBRE1 NOT IN('No Disponible', 'No disponible', 'N A') OR (LCC.CLIE_CLIE_NOMBRE1 = 'No Disponible' AND LCC.CLIE_CLIE_FNAC = '1919-01-01') )
    AND SUB.SUBCLASE = '{subclase_snow}'
    AND FFM.SOCI_SOCI_ID NOT IN (SELECT * FROM COMPRARON)
'''

segm_2 = pd.DataFrame()
for i in range(len(articulos_destacados_2)):
    cursor.execute(query.format(
       orin_snow = articulos_destacados_2.iloc[i]['ORIN'],
       subclase_snow = articulos_destacados_2.iloc[i]['SUBCLASE']))
    df_aux = cursor.fetch_pandas_all()
    df_aux['MARCA'] = articulos_destacados_2.iloc[i]['MARCA']
    df_aux['ORIN'] = articulos_destacados_2.iloc[i]['ORIN']
    df_aux['ARTICULO'] = articulos_destacados_2.iloc[i]['ARTICULO']
    segm_2 = pd.concat([segm_2, df_aux])

segm_2 = segm_2[['SUB_NAME', 'SUBCLASE', 'MARCA', 'ORIN', 'ARTICULO',
       'SOCI_SOCI_ID', 'CLIE_TIPO_DOC', 'CLIE_CLIE_NDOC', 'ULTIMOS_4',
       'CELULAR']]
print(segm_2.shape[0])
segm_2.head(2)

776942


,SUB_NAME,SUBCLASE,MARCA,ORIN,ARTICULO,SOCI_SOCI_ID,CLIE_TIPO_DOC,CLIE_CLIE_NDOC,ULTIMOS_4,CELULAR
0,HELADOS 2 LITROS,3200010002,CRUFI,1000099233,HELADO CRUFI 2.00 L SAMBAYON/CHOCOLATE/D,1065332,C,63368637,CI:8637,94550813
1,HELADOS 2 LITROS,3200010002,CRUFI,1000099233,HELADO CRUFI 2.00 L SAMBAYON/CHOCOLATE/D,464063,C,11996092,CI:6092,96650173


In [12]:
segm_2[segm_2.duplicated(subset = ['SOCI_SOCI_ID', 'ORIN'], keep = False)]

,SUB_NAME,SUBCLASE,MARCA,ORIN,ARTICULO,SOCI_SOCI_ID,CLIE_TIPO_DOC,CLIE_CLIE_NDOC,ULTIMOS_4,CELULAR


#### 2.2. Clientes que compran la subclase pero no el articulo (Score 4 o 5 en la subclase)

In [13]:
query = '''
WITH COMPRARON AS
    (
    SELECT
        DISTINCT
        FFM.SOCI_SOCI_ID
    FROM
        MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA
        INNER JOIN MSTRDB.DWH.FT_VENTAS AS FV ON FV.ARTC_ARTC_ID = LAA.ARTC_ARTC_ID
        INNER JOIN MSTRDB.DWH.FT_FDLN_MOVIMIENTOS AS FFM ON FFM.TICKET = FV.TICKET
    WHERE
        FFM.TIEM_DIA_ID >= DATEADD(MONTH, -1, CURRENT_DATE)
        AND LAA.ORIN = '{orin_snow}'
    ),

BZM AS
    (
    SELECT
        DISTINCT SOCI_SOCI_ID
    FROM
        BIZMETRIKS.DWH.LU_SCORING_SUBCLASE
    WHERE
        SCORE > 3
        AND SUBCLASE = '{subclase_snow}'
    )

SELECT
    DISTINCT
    SUB.SUB_NAME,
    SUB.SUBCLASE,
    CONT.SOCI_SOCI_ID,
    CONT.CLIE_TIPO_DOC,
    CONT.CLIE_CLIE_NDOC,
    CASE
        WHEN CONT.CLIE_TIPO_DOC LIKE 'C' THEN 'CI:' || RIGHT(CONT.CLIE_CLIE_NDOC, 4)
        ELSE 'Doc.' || RIGHT(CONT.CLIE_CLIE_NDOC, 4)
        END AS ULTIMOS_4,
    CONT.CELULAR
FROM
    MSTRDB.DWH.LU_ARTC_BRAND AS LAB
    INNER JOIN MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA ON LAA.ARTC_BRAND_ID = LAB.ARTC_BRAND_ID
    INNER JOIN MSTRDB.DWH.ITEM_MASTER AS IM ON IM.ITEM = LAA.ORIN
    INNER JOIN MSTRDB.DWH.SUBCLASS AS SUB ON SUB.SUBCLASE = IM.SUBCLASE
    INNER JOIN MSTRDB.DWH.FT_VENTAS AS FV ON FV.ARTC_ARTC_ID = LAA.ARTC_ARTC_ID
    INNER JOIN MSTRDB.DWH.FT_FDLN_MOVIMIENTOS AS FFM ON FFM.TICKET = FV.TICKET
    INNER JOIN SANDBOX_PLUS.DWH.CONTACTABLES_2 AS CONT ON CONT.SOCI_SOCI_ID = FFM.SOCI_SOCI_ID
    INNER JOIN MSTRDB.DWH.LU_CLIE_CLIENTE AS LCC ON LCC.SOCI_SOCI_ID = FFM.SOCI_SOCI_ID
WHERE
    FFM.TIEM_DIA_ID >= DATEADD(MONTH, -1, CURRENT_DATE)
    AND CONT.CELULAR IS NOT NULL
    AND CONT.WP = 1
    AND CONT.SOCI_SOCI_ID NOT IN (SELECT SOCI_SOCI_ID FROM SANDBOX_PLUS.DWH.SOCIOS_NO_COMUNICAR)
    AND ( LCC.CLIE_CLIE_NOMBRE1 NOT IN('No Disponible', 'No disponible', 'N A') OR (LCC.CLIE_CLIE_NOMBRE1 = 'No Disponible' AND LCC.CLIE_CLIE_FNAC = '1919-01-01') )
    AND SUB.SUBCLASE = '{subclase_snow}'
    AND FFM.SOCI_SOCI_ID NOT IN (SELECT * FROM COMPRARON)
    AND FFM.SOCI_SOCI_ID IN (SELECT * FROM BZM)
'''

segm_2_2 = pd.DataFrame()
for i in range(len(articulos_destacados_2)):
    cursor.execute(query.format(
       orin_snow = articulos_destacados_2.iloc[i]['ORIN'],
       subclase_snow = articulos_destacados_2.iloc[i]['SUBCLASE']))
    df_aux = cursor.fetch_pandas_all()
    df_aux['MARCA'] = articulos_destacados_2.iloc[i]['MARCA']
    df_aux['ORIN'] = articulos_destacados_2.iloc[i]['ORIN']
    df_aux['ARTICULO'] = articulos_destacados_2.iloc[i]['ARTICULO']
    segm_2_2 = pd.concat([segm_2_2, df_aux])

segm_2_2 = segm_2_2[['SUB_NAME', 'SUBCLASE', 'MARCA', 'ORIN', 'ARTICULO',
       'SOCI_SOCI_ID', 'CLIE_TIPO_DOC', 'CLIE_CLIE_NDOC', 'ULTIMOS_4',
       'CELULAR']]
print(segm_2_2.shape[0])
segm_2_2.head(2)

KeyboardInterrupt: 

In [ ]:
segm_2_2[segm_2_2.duplicated(subset = ['SOCI_SOCI_ID', 'ORIN'], keep = False)]

#### 3.1. Clientes activos que no compran la subclase y tienen score 5

In [32]:
query = '''
WITH COMPRARON AS
    (
    SELECT
        DISTINCT
        FFM.SOCI_SOCI_ID
    FROM
        MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA
        INNER JOIN MSTRDB.DWH.FT_VENTAS AS FV ON FV.ARTC_ARTC_ID = LAA.ARTC_ARTC_ID
        INNER JOIN MSTRDB.DWH.FT_FDLN_MOVIMIENTOS AS FFM ON FFM.TICKET = FV.TICKET
        INNER JOIN MSTRDB.DWH.ITEM_MASTER AS IM ON IM.ITEM = LAA.ORIN
    WHERE
        FFM.TIEM_DIA_ID >= DATEADD(MONTH, -1, CURRENT_DATE)
        AND IM.SUBCLASE = '{subclase_snow}'
    ),

ACTIVOS AS
    (
    SELECT
        DISTINCT SOCI_SOCI_ID
    FROM
        MSTRDB.DWH.FT_FDLN_MOVIMIENTOS
    WHERE
        TIEM_DIA_ID >= DATEADD(MONTH, -3, CURRENT_DATE)
        AND GEOG_LOCL_ID IN (SELECT GEOG_LOCL_ID FROM MSTRDB.DWH.LU_GEOG_LOCAL WHERE GEOG_UNNG_ID = 2)
    ),

BZM AS
    (
    SELECT
        DISTINCT SOCI_SOCI_ID
    FROM
        BIZMETRIKS.DWH.LU_SCORING_SUBCLASE
    WHERE
        SCORE > 4
        AND SUBCLASE = '{subclase_snow}'
    )

SELECT
    CONT.SOCI_SOCI_ID,
    CONT.CLIE_TIPO_DOC,
    CONT.CLIE_CLIE_NDOC,
    CASE
        WHEN CONT.CLIE_TIPO_DOC LIKE 'C' THEN 'CI:' || RIGHT(CONT.CLIE_CLIE_NDOC, 4)
        ELSE 'Doc.' || RIGHT(CONT.CLIE_CLIE_NDOC, 4)
        END AS ULTIMOS_4,
    CONT.CELULAR,
    CONT.CORREO,
    CONT.WP
FROM
    SANDBOX_PLUS.DWH.CONTACTABLES_2 AS CONT
    INNER JOIN MSTRDB.DWH.LU_CLIE_CLIENTE AS LCC ON LCC.SOCI_SOCI_ID = CONT.SOCI_SOCI_ID
WHERE
    (CONT.CELULAR IS NOT NULL OR CONT.CORREO IS NOT NULL)
    AND CONT.SOCI_SOCI_ID NOT IN (SELECT SOCI_SOCI_ID FROM SANDBOX_PLUS.DWH.SOCIOS_NO_COMUNICAR)
    AND ( LCC.CLIE_CLIE_NOMBRE1 NOT IN('No Disponible', 'No disponible', 'N A') OR (LCC.CLIE_CLIE_NOMBRE1 = 'No Disponible' AND LCC.CLIE_CLIE_FNAC = '1919-01-01') )
    AND CONT.SOCI_SOCI_ID NOT IN (SELECT * FROM COMPRARON)
    AND CONT.SOCI_SOCI_ID IN (SELECT * FROM BZM)
    AND CONT.SOCI_SOCI_ID IN (SELECT * FROM ACTIVOS)
'''

segm_3 = pd.DataFrame()
marca_subclase_unique = articulos_destacados_2[['SUBCLASE', 'SUB_NAME']].drop_duplicates()
for i in range(len(marca_subclase_unique)):
    cursor.execute(query.format(
       subclase_snow = marca_subclase_unique.iloc[i]['SUBCLASE']))
    df_aux = cursor.fetch_pandas_all()
    df_aux['SUB_NAME'] = marca_subclase_unique.iloc[i]['SUB_NAME']
    df_aux['SUBCLASE'] = marca_subclase_unique.iloc[i]['SUBCLASE']
    segm_3 = pd.concat([segm_3, df_aux])

segm_3 = segm_3.merge(articulos_destacados_2[['SUBCLASE', 'MARCA']].drop_duplicates(), on = 'SUBCLASE', how = 'inner')

segm_3 = segm_3[['SUB_NAME', 'SUBCLASE', 'MARCA', 'SOCI_SOCI_ID', 'CLIE_TIPO_DOC', 'CLIE_CLIE_NDOC', 'ULTIMOS_4', 'CELULAR', 'CORREO', 'WP']]
print(segm_3.shape[0])
segm_3.head(2)

1442018


,SUB_NAME,SUBCLASE,MARCA,SOCI_SOCI_ID,CLIE_TIPO_DOC,CLIE_CLIE_NDOC,ULTIMOS_4,CELULAR,CORREO,WP
0,HELADOS 2 LITROS,3200010002,CRUFI,1435874,C,51992781,CI:2781,94324545,None,1
1,HELADOS 2 LITROS,3200010002,CRUFI,1435899,C,51095434,CI:5434,92500747,None,1


In [36]:
# Para Celular, la comunicacion sera por WP
# Entonces si no tienen WP, elimino el Celular

segm_3['CELULAR'][segm_3['WP'] == 0] = np.nan
segm_3['CELULAR'][segm_3['WP'] == 0].value_counts()

Series([], Name: CELULAR, dtype: int64)

In [42]:
for i in segm_3['SUB_NAME'].unique():
    print(i)
    df_aux = segm_3[segm_3['SUB_NAME'] == i]
    df_aux.drop('WP', axis = 1).to_csv(f"Segm 3.1 - Subclase {i}.csv", index = False)

HELADOS 2 LITROS
YOGURES BEBIBLES ENTERO
PACKS
JABONES DETERGENTE LIQUIDOS PARA ROPA
PAPAS FRITAS EN BOLSA
PAPAS CONGELADAS
PAPEL HIGIENICO HOJA DOBLE Y TRIPLE
LECHE ESPECIALES
CHOCOLATADA
PANALES VALUE
QUESO SANDWICH AL CORTE
CERVEZAS NACIONALES
JAMON COCIDO AL CORTE


In [15]:
segm_3[segm_3.duplicated(subset = ['SOCI_SOCI_ID', 'SUBCLASE'], keep = False)]
segm_3[segm_3.duplicated(subset = ['SOCI_SOCI_ID', 'MARCA'], keep = False)].groupby(['SUB_NAME', 'MARCA'])['SUBCLASE'].count().reset_index()

,SUB_NAME,MARCA,SUBCLASE
0,CHOCOLATADA,PARMALAT,80587
1,LECHE ESPECIALES,PARMALAT,80587


#### 3.2. Clientes activos que no compran la subclase y tienen score 4 o 5

In [ ]:
query = '''
WITH COMPRARON AS
    (
    SELECT
        DISTINCT
        FFM.SOCI_SOCI_ID
    FROM
        MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA
        INNER JOIN MSTRDB.DWH.FT_VENTAS AS FV ON FV.ARTC_ARTC_ID = LAA.ARTC_ARTC_ID
        INNER JOIN MSTRDB.DWH.FT_FDLN_MOVIMIENTOS AS FFM ON FFM.TICKET = FV.TICKET
        INNER JOIN MSTRDB.DWH.ITEM_MASTER AS IM ON IM.ITEM = LAA.ORIN
    WHERE
        FFM.TIEM_DIA_ID >= DATEADD(MONTH, -1, CURRENT_DATE)
        AND IM.SUBCLASE = '{subclase_snow}'
    ),

ACTIVOS AS
    (
    SELECT
        DISTINCT SOCI_SOCI_ID
    FROM
        MSTRDB.DWH.FT_FDLN_MOVIMIENTOS
    WHERE
        TIEM_DIA_ID >= DATEADD(MONTH, -3, CURRENT_DATE)
        AND GEOG_LOCL_ID IN (SELECT GEOG_LOCL_ID FROM MSTRDB.DWH.LU_GEOG_LOCAL WHERE GEOG_UNNG_ID = 2)
    ),

BZM AS
    (
    SELECT
        DISTINCT SOCI_SOCI_ID
    FROM
        BIZMETRIKS.DWH.LU_SCORING_SUBCLASE
    WHERE
        SCORE > 3
        AND SUBCLASE = '{subclase_snow}'
    )

SELECT
    CONT.SOCI_SOCI_ID,
    CONT.CLIE_TIPO_DOC,
    CONT.CLIE_CLIE_NDOC,
    CASE
        WHEN CONT.CLIE_TIPO_DOC LIKE 'C' THEN 'CI:' || RIGHT(CONT.CLIE_CLIE_NDOC, 4)
        ELSE 'Doc.' || RIGHT(CONT.CLIE_CLIE_NDOC, 4)
        END AS ULTIMOS_4,
    CONT.CELULAR
FROM
    SANDBOX_PLUS.DWH.CONTACTABLES_2 AS CONT
    INNER JOIN MSTRDB.DWH.LU_CLIE_CLIENTE AS LCC ON LCC.SOCI_SOCI_ID = CONT.SOCI_SOCI_ID
WHERE
    CONT.CELULAR IS NOT NULL
    AND CONT.WP = 1
    AND CONT.SOCI_SOCI_ID NOT IN (SELECT SOCI_SOCI_ID FROM SANDBOX_PLUS.DWH.SOCIOS_NO_COMUNICAR)
    AND ( LCC.CLIE_CLIE_NOMBRE1 NOT IN('No Disponible', 'No disponible', 'N A') OR (LCC.CLIE_CLIE_NOMBRE1 = 'No Disponible' AND LCC.CLIE_CLIE_FNAC = '1919-01-01') )
    AND CONT.SOCI_SOCI_ID NOT IN (SELECT * FROM COMPRARON)
    AND CONT.SOCI_SOCI_ID IN (SELECT * FROM BZM)
    AND CONT.SOCI_SOCI_ID IN (SELECT * FROM ACTIVOS)
'''

segm_3_2 = pd.DataFrame()
marca_subclase_unique = articulos_destacados_2[['SUBCLASE', 'SUB_NAME']].drop_duplicates()
for i in range(len(marca_subclase_unique)):
    cursor.execute(query.format(
       subclase_snow = marca_subclase_unique.iloc[i]['SUBCLASE']))
    df_aux = cursor.fetch_pandas_all()
    df_aux['SUB_NAME'] = marca_subclase_unique.iloc[i]['SUB_NAME']
    df_aux['SUBCLASE'] = marca_subclase_unique.iloc[i]['SUBCLASE']
    segm_3_2 = pd.concat([segm_3_2, df_aux])

segm_3_2 = segm_3_2.merge(articulos_destacados_2[['SUBCLASE', 'MARCA']].drop_duplicates(), on = 'SUBCLASE', how = 'inner')

segm_3_2 = segm_3_2[['SUB_NAME', 'SUBCLASE', 'MARCA', 'SOCI_SOCI_ID', 'CLIE_TIPO_DOC', 'CLIE_CLIE_NDOC', 'ULTIMOS_4', 'CELULAR']]
print(segm_3_2.shape[0])
segm_3_2.head(2)

In [ ]:
segm_3_2[segm_3_2.duplicated(subset = ['SOCI_SOCI_ID', 'SUBCLASE'], keep = False)]
segm_3_2[segm_3_2.duplicated(subset = ['SOCI_SOCI_ID', 'MARCA'], keep = False)].groupby(['SUB_NAME', 'MARCA'])['SUBCLASE'].count().reset_index()

#### 4. Clientes que compran la subclase pero no la marca

In [ ]:
query = '''
WITH COMPRARON AS
    (
    SELECT
        DISTINCT
        FFM.SOCI_SOCI_ID
    FROM
        MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA
        INNER JOIN MSTRDB.DWH.LU_ARTC_BRAND AS LAB ON LAA.ARTC_BRAND_ID = LAB.ARTC_BRAND_ID
        INNER JOIN MSTRDB.DWH.FT_VENTAS AS FV ON FV.ARTC_ARTC_ID = LAA.ARTC_ARTC_ID
        INNER JOIN MSTRDB.DWH.FT_FDLN_MOVIMIENTOS AS FFM ON FFM.TICKET = FV.TICKET
    WHERE
        FFM.TIEM_DIA_ID >= DATEADD(MONTH, -1, CURRENT_DATE)
        AND LAB.ARTC_BRAND_DESC = '{marca_snow}'
    )

SELECT
    DISTINCT
    SUB.SUB_NAME,
    SUB.SUBCLASE,
    CONT.SOCI_SOCI_ID,
    CONT.CLIE_TIPO_DOC,
    CONT.CLIE_CLIE_NDOC,
    CASE
        WHEN CONT.CLIE_TIPO_DOC LIKE 'C' THEN 'CI:' || RIGHT(CONT.CLIE_CLIE_NDOC, 4)
        ELSE 'Doc.' || RIGHT(CONT.CLIE_CLIE_NDOC, 4)
        END AS ULTIMOS_4,
    CONT.CELULAR
FROM
    MSTRDB.DWH.LU_ARTC_BRAND AS LAB
    INNER JOIN MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA ON LAA.ARTC_BRAND_ID = LAB.ARTC_BRAND_ID
    INNER JOIN MSTRDB.DWH.ITEM_MASTER AS IM ON IM.ITEM = LAA.ORIN
    INNER JOIN MSTRDB.DWH.SUBCLASS AS SUB ON SUB.SUBCLASE = IM.SUBCLASE
    INNER JOIN MSTRDB.DWH.FT_VENTAS AS FV ON FV.ARTC_ARTC_ID = LAA.ARTC_ARTC_ID
    INNER JOIN MSTRDB.DWH.FT_FDLN_MOVIMIENTOS AS FFM ON FFM.TICKET = FV.TICKET
    INNER JOIN SANDBOX_PLUS.DWH.CONTACTABLES_2 AS CONT ON CONT.SOCI_SOCI_ID = FFM.SOCI_SOCI_ID
    INNER JOIN MSTRDB.DWH.LU_CLIE_CLIENTE AS LCC ON LCC.SOCI_SOCI_ID = FFM.SOCI_SOCI_ID
WHERE
    FFM.TIEM_DIA_ID >= DATEADD(MONTH, -1, CURRENT_DATE)
    AND CONT.CELULAR IS NOT NULL
    AND CONT.WP = 1
    AND CONT.SOCI_SOCI_ID NOT IN (SELECT SOCI_SOCI_ID FROM SANDBOX_PLUS.DWH.SOCIOS_NO_COMUNICAR)
    AND ( LCC.CLIE_CLIE_NOMBRE1 NOT IN('No Disponible', 'No disponible', 'N A') OR (LCC.CLIE_CLIE_NOMBRE1 = 'No Disponible' AND LCC.CLIE_CLIE_FNAC = '1919-01-01') )
    AND SUB.SUBCLASE = {subclase_snow}
    AND FFM.SOCI_SOCI_ID NOT IN (SELECT * FROM COMPRARON)
'''

segm_4 = pd.DataFrame()
marca_subclase_unique = articulos_destacados_2[['MARCA', 'SUBCLASE']].drop_duplicates()
for i in range(len(marca_subclase_unique)):
    cursor.execute(query.format(
       marca_snow = marca_subclase_unique.iloc[i]['MARCA'],
       subclase_snow = marca_subclase_unique.iloc[i]['SUBCLASE']))
    df_aux = cursor.fetch_pandas_all()
    df_aux['MARCA'] = marca_subclase_unique.iloc[i]['MARCA']
    segm_4 = pd.concat([segm_4, df_aux])

segm_4 = segm_4[['SUB_NAME', 'SUBCLASE', 'MARCA', 'SOCI_SOCI_ID', 'CLIE_TIPO_DOC', 'CLIE_CLIE_NDOC', 'ULTIMOS_4', 'CELULAR']]
print(segm_4.shape[0])
segm_4.head(2)

In [ ]:
segm_4[segm_4.duplicated(subset = ['SOCI_SOCI_ID', 'SUBCLASE'], keep = False)]
segm_4[segm_4.duplicated(subset = ['SOCI_SOCI_ID', 'MARCA'], keep = False)].groupby(['SUB_NAME', 'MARCA'])['SUBCLASE'].count().reset_index()

#### 5. Clientes que en el ultimo mes no vengan a tata y tengan Score 5 en la subclase

In [39]:
query = '''
WITH ACTIVOS AS
    (
    SELECT
        DISTINCT SOCI_SOCI_ID
    FROM
        MSTRDB.DWH.FT_FDLN_MOVIMIENTOS
    WHERE
        TIEM_DIA_ID >= DATEADD(MONTH, -1, CURRENT_DATE)
        AND GEOG_LOCL_ID IN (SELECT GEOG_LOCL_ID FROM MSTRDB.DWH.LU_GEOG_LOCAL WHERE GEOG_UNNG_ID = 2)
    ),

BZM AS
    (
    SELECT
        DISTINCT SOCI_SOCI_ID
    FROM
        BIZMETRIKS.DWH.LU_SCORING_SUBCLASE
    WHERE
        SCORE > 4
        AND SUBCLASE = '{subclase_snow}'
    )

SELECT
    CONT.SOCI_SOCI_ID,
    CONT.CLIE_TIPO_DOC,
    CONT.CLIE_CLIE_NDOC,
    CASE
        WHEN CONT.CLIE_TIPO_DOC LIKE 'C' THEN 'CI:' || RIGHT(CONT.CLIE_CLIE_NDOC, 4)
        ELSE 'Doc.' || RIGHT(CONT.CLIE_CLIE_NDOC, 4)
        END AS ULTIMOS_4,
    CONT.CELULAR,
    CONT.CORREO,
    CONT.WP
FROM
    SANDBOX_PLUS.DWH.CONTACTABLES_2 AS CONT
    INNER JOIN MSTRDB.DWH.LU_CLIE_CLIENTE AS LCC ON LCC.SOCI_SOCI_ID = CONT.SOCI_SOCI_ID
WHERE
    (CONT.CELULAR IS NOT NULL OR CONT.CORREO IS NOT NULL)
    AND CONT.WP = 1
    AND CONT.SOCI_SOCI_ID NOT IN (SELECT SOCI_SOCI_ID FROM SANDBOX_PLUS.DWH.SOCIOS_NO_COMUNICAR)
    AND ( LCC.CLIE_CLIE_NOMBRE1 NOT IN('No Disponible', 'No disponible', 'N A') OR (LCC.CLIE_CLIE_NOMBRE1 = 'No Disponible' AND LCC.CLIE_CLIE_FNAC = '1919-01-01') )
    AND CONT.SOCI_SOCI_ID IN (SELECT * FROM BZM)
    AND CONT.SOCI_SOCI_ID NOT IN (SELECT * FROM ACTIVOS)
'''

segm_5 = pd.DataFrame()
marca_subclase_unique = articulos_destacados_2[['SUBCLASE', 'SUB_NAME']].drop_duplicates()
for i in range(len(marca_subclase_unique)):
    cursor.execute(query.format(
       subclase_snow = marca_subclase_unique.iloc[i]['SUBCLASE']))
    df_aux = cursor.fetch_pandas_all()
    df_aux['SUB_NAME'] = marca_subclase_unique.iloc[i]['SUB_NAME']
    df_aux['SUBCLASE'] = marca_subclase_unique.iloc[i]['SUBCLASE']
    segm_5 = pd.concat([segm_5, df_aux])

segm_5 = segm_5.merge(articulos_destacados_2[['SUBCLASE', 'MARCA']].drop_duplicates(), on = 'SUBCLASE', how = 'inner')

segm_5 = segm_5[['SUB_NAME', 'SUBCLASE', 'MARCA', 'SOCI_SOCI_ID', 'CLIE_TIPO_DOC', 'CLIE_CLIE_NDOC', 'ULTIMOS_4', 'CELULAR', 'CORREO', 'WP']]
print(segm_5.shape[0])
segm_5.head(2)

480196


,SUB_NAME,SUBCLASE,MARCA,SOCI_SOCI_ID,CLIE_TIPO_DOC,CLIE_CLIE_NDOC,ULTIMOS_4,CELULAR,CORREO,WP
0,HELADOS 2 LITROS,3200010002,CRUFI,424169,C,33913931,CI:3931,99361891,None,1
1,HELADOS 2 LITROS,3200010002,CRUFI,359467,C,33947099,CI:7099,99481357,None,1


In [40]:
# Para Celular, la comunicacion sera por WP
# Entonces si no tienen WP, elimino el Celular

segm_5['CELULAR'][segm_5['WP'] == 0] = np.nan
segm_5['CELULAR'][segm_5['WP'] == 0].value_counts()

Series([], Name: CELULAR, dtype: int64)

In [41]:
segm_5.drop('WP', axis = 1).to_csv(f"Segm 5.csv", index = False)

In [17]:
segm_5[segm_5.duplicated(subset = ['SOCI_SOCI_ID', 'SUBCLASE'], keep = False)]
segm_5[segm_5.duplicated(subset = ['SOCI_SOCI_ID', 'MARCA'], keep = False)].groupby(['SUB_NAME', 'MARCA'])['SUBCLASE'].count().reset_index()

,SUB_NAME,MARCA,SUBCLASE
0,CHOCOLATADA,PARMALAT,29313
1,LECHE ESPECIALES,PARMALAT,29313


In [23]:
segm_5.head()

,SUB_NAME,SUBCLASE,MARCA,SOCI_SOCI_ID,CLIE_TIPO_DOC,CLIE_CLIE_NDOC,ULTIMOS_4,CELULAR
0,HELADOS 2 LITROS,3200010002,CRUFI,156406,C,36818233,CI:8233,94602080
1,HELADOS 2 LITROS,3200010002,CRUFI,282173,C,36847391,CI:7391,99705987
2,HELADOS 2 LITROS,3200010002,CRUFI,759016,C,37013824,CI:3824,99983618
3,HELADOS 2 LITROS,3200010002,CRUFI,177860,C,37014210,CI:4210,92402845
4,HELADOS 2 LITROS,3200010002,CRUFI,150063,C,36817320,CI:7320,99731552


### Resumen

#### Segmento 1, 2.1 y 2.2

In [75]:
resumen_segmentos_1 = articulos_destacados_2.copy(deep = True)
segm_1_resumen = segm_1.groupby('ORIN')['SOCI_SOCI_ID'].nunique().reset_index()
resumen_segmentos_1 = resumen_segmentos_1.merge(segm_1_resumen, on = 'ORIN', how = 'inner')
resumen_segmentos_1.rename({'SOCI_SOCI_ID':'Clientes ya compran el articulo'}, axis = 1, inplace = True)

segm_2_resumen = segm_2.groupby('ORIN')['SOCI_SOCI_ID'].nunique().reset_index()
resumen_segmentos_1 = resumen_segmentos_1.merge(segm_2_resumen, on = 'ORIN', how = 'inner')
resumen_segmentos_1.rename({'SOCI_SOCI_ID':'Clientes ya compran la subclase pero no el articulo'}, axis = 1, inplace = True)

segm_2_2_resumen = segm_2_2.groupby('ORIN')['SOCI_SOCI_ID'].nunique().reset_index()
resumen_segmentos_1 = resumen_segmentos_1.merge(segm_2_2_resumen, on = 'ORIN', how = 'inner')
resumen_segmentos_1.rename({'SOCI_SOCI_ID':'Clientes ya compran la subclase pero no el articulo, Score 4 y 5'}, axis = 1, inplace = True)

resumen_segmentos_1

,SUB_NAME,SUBCLASE,MARCA,ORIN,ARTICULO,Clientes ya compran el articulo,Clientes ya compran la subclase pero no el articulo,"Clientes ya compran la subclase pero no el articulo, Score 4 y 5"
0,PACKS,1900010003,PORTEZUELO,1000054863,ALFAJOR CHOCOLATE PORTEZUELO PACK 6.00 U.,3671,17324,12171
1,PACKS,1900010003,PORTEZUELO,1000054862,ALFAJOR NIEVE PORTEZUELO PACK X 6 U,2209,18786,13256
2,JABONES DETERGENTE LIQUIDOS PARA ROPA,8800030003,NEVEX,1000455579,NEVEX MATIC LIQUIDO DOY PACK 3L PARA DILUIR MA...,338,29472,21075
3,PAPAS FRITAS EN BOLSA,2500050001,LAYS,1000053389,PAPAS FRITAS LAY'S CLASICAS 400.00 G 1,5818,59609,39024
4,HELADOS 2 LITROS,3200010002,CRUFI,1000099233,HELADO CRUFI 2.00 L SAMBAYON/CHOCOLATE/D,757,11240,8071
5,HELADOS 2 LITROS,3200010002,CRUFI,1000099234,HELADO CRUFI 2.00 L CREMA VETEADA,629,11368,8153
6,HELADOS 2 LITROS,3200010002,CRUFI,1000099229,HELADO CRUFI 2.00 L CREMA/CHOCOLATE/FRUT,897,11100,7982
7,HELADOS 2 LITROS,3200010002,CRUFI,1000099231,HELADO CRUFI 2.00 L FLAN C/DULCE DE LECH,929,11068,7917
8,YOGURES BEBIBLES ENTERO,7900010002,BIOTOP,1000103474,YOGUR BEBIBLE BIOTOP SACHET 1.00 L FRUT.LIGHT,4837,60455,43193
9,YOGURES BEBIBLES ENTERO,7900010002,BIOTOP,1000103473,YOGUR BEBIBLE BIOTOP SACHET 1.00 L DURAZNO,5515,59777,42755


#### Segmento 3.1 y 3.2

In [17]:
resumen_segmentos_2 = articulos_destacados_2.copy(deep = True)
resumen_segmentos_2.drop(['ORIN', 'ARTICULO'], axis = 1, inplace = True)
resumen_segmentos_2.drop_duplicates(inplace = True)

segm_3_resumen = segm_3.groupby('SUBCLASE')['SOCI_SOCI_ID'].nunique().reset_index()
resumen_segmentos_2 = resumen_segmentos_2.merge(segm_3_resumen, on = 'SUBCLASE', how = 'inner')
resumen_segmentos_2.rename({'SOCI_SOCI_ID':'Clientes que no compran la subclase y tienen Score 5'}, axis = 1, inplace = True)

segm_3_2_resumen = segm_3_2.groupby('SUBCLASE')['SOCI_SOCI_ID'].nunique().reset_index()
resumen_segmentos_2 = resumen_segmentos_2.merge(segm_3_2_resumen, on = 'SUBCLASE', how = 'inner')
resumen_segmentos_2.rename({'SOCI_SOCI_ID':'Clientes que no compran la subclase y tienen Score 4 o 5'}, axis = 1, inplace = True)

resumen_segmentos_2

,SUB_NAME,SUBCLASE,MARCA,Clientes que no compran la subclase y tienen Score 5,Clientes que no compran la subclase y tienen Score 4 o 5
0,PAPAS FRITAS EN BOLSA,2500050001,LAYS,97368,209932
1,HELADOS 2 LITROS,3200010002,CRUFI,116266,241377
2,YOGURES BEBIBLES ENTERO,7900010002,BIOTOP,94918,206529
3,PACKS,1900010003,PORTEZUELO,114632,237943
4,JABONES DETERGENTE LIQUIDOS PARA ROPA,8800030003,NEVEX,107701,229316
5,QUESO SANDWICH AL CORTE,4200020006,COLONIAL,113559,233375
6,JAMON COCIDO AL CORTE,4000030002,OTTONELLO,101741,218298
7,CERVEZAS NACIONALES,100020003,ZILLERTAL,114943,235863
8,PAPAS CONGELADAS,3000020002,MC CAIN,112913,236181
9,PAPEL HIGIENICO HOJA DOBLE Y TRIPLE,9100030001,ELITE,111093,233729


#### Segmento 4

In [102]:
resumen_segmentos_3 = articulos_destacados_2.copy(deep = True)
resumen_segmentos_3.drop(['ORIN', 'ARTICULO'], axis = 1, inplace = True)
resumen_segmentos_3.drop_duplicates(inplace = True)

segm_4_resumen = segm_4.groupby('SUBCLASE')['SOCI_SOCI_ID'].nunique().reset_index()
resumen_segmentos_3 = resumen_segmentos_3.merge(segm_4_resumen, on = 'SUBCLASE', how = 'inner')
resumen_segmentos_3.rename({'SOCI_SOCI_ID':'Clientes que compran la subclase pero no la marca'}, axis = 1, inplace = True)
resumen_segmentos_3

,SUB_NAME,SUBCLASE,MARCA,Clientes que no compran la subclase pero no la marca
0,PACKS,1900010003,PORTEZUELO,5047
1,JABONES DETERGENTE LIQUIDOS PARA ROPA,8800030003,NEVEX,24116
2,PAPAS FRITAS EN BOLSA,2500050001,LAYS,34178
3,HELADOS 2 LITROS,3200010002,CRUFI,7115
4,YOGURES BEBIBLES ENTERO,7900010002,BIOTOP,38403
5,PAPAS CONGELADAS,3000020002,MC CAIN,11325
6,PAPEL HIGIENICO HOJA DOBLE Y TRIPLE,9100030001,ELITE,19505
7,LECHE ESPECIALES,7600060004,PARMALAT,7802
8,CHOCOLATADA,7600030001,PARMALAT,28558
9,PANALES VALUE,9800010003,BABYSEC,1021


#### Segmento 5

In [25]:
resumen_segmentos_4 = articulos_destacados_2.copy(deep = True)
resumen_segmentos_4.drop(['ORIN', 'ARTICULO'], axis = 1, inplace = True)
resumen_segmentos_4.drop_duplicates(inplace = True)

segm_5_resumen = segm_5.groupby('SUBCLASE')['SOCI_SOCI_ID'].nunique().reset_index()
resumen_segmentos_4 = resumen_segmentos_4.merge(segm_5_resumen, on = 'SUBCLASE', how = 'inner')
resumen_segmentos_4.rename({'SOCI_SOCI_ID':'Clientes que no compran en TATA en ultimo Mes y tienen Score 5 en la subclase'}, axis = 1, inplace = True)
resumen_segmentos_4

,SUB_NAME,SUBCLASE,MARCA,Clientes que no compran en TATA en ultimo Mes y tienen Score 5 en la subclase
0,PAPAS FRITAS EN BOLSA,2500050001,LAYS,37895
1,HELADOS 2 LITROS,3200010002,CRUFI,41754
2,YOGURES BEBIBLES ENTERO,7900010002,BIOTOP,36133
3,PACKS,1900010003,PORTEZUELO,38368
4,JABONES DETERGENTE LIQUIDOS PARA ROPA,8800030003,NEVEX,41252
5,QUESO SANDWICH AL CORTE,4200020006,COLONIAL,34632
6,JAMON COCIDO AL CORTE,4000030002,OTTONELLO,34354
7,CERVEZAS NACIONALES,100020003,ZILLERTAL,37816
8,PAPAS CONGELADAS,3000020002,MC CAIN,38342
9,PAPEL HIGIENICO HOJA DOBLE Y TRIPLE,9100030001,ELITE,39779


In [26]:
resumen_segmentos_4.to_excel('s.xlsx', index = False)

# Excel

In [111]:
# Segmentos Fede

with pd.ExcelWriter(
        f"C:\\Users\\leonardo.mangold\\PycharmProjects\\analisis\\kpis_promos_mauri\\Entregas\\Articulos Destacados {datetime.today().strftime('%Y-%m-%d')}.xlsx",
        engine='xlsxwriter') as writer:

    resumen_segmentos_1.to_excel(writer, sheet_name=f"Resumen Segmentos 1, 2.1 y 2.2", index=False)
    segm_1.to_excel(writer, sheet_name=f"1 Compran el art", index=False)
    segm_2.to_excel(writer, sheet_name=f"2.1 Compran subcl, no art", index=False)
    segm_2_2.to_excel(writer, sheet_name=f"2.2 Compran subcl, no art, S>3", index=False)
    resumen_segmentos_2.to_excel(writer, sheet_name=f"Resumen Segmento 3", index=False)
    #segm_3.to_excel(writer, sheet_name=f"3. No compran subcl, S>5", index=False)
    resumen_segmentos_3.to_excel(writer, sheet_name=f"Resumen Segmento 4", index=False)
    segm_4.to_excel(writer, sheet_name=f"4. Compran subcl pero no marca", index=False)

In [ ]:
# Mauri

with pd.ExcelWriter(
        f"C:\\Users\\leonardo.mangold\\PycharmProjects\\analisis\\kpis_promos_mauri\\Entregas\\Promos {datetime.today().strftime('%Y-%m-%d')}.xlsx",
        engine='xlsxwriter') as writer:



    precios.to_excel(writer, sheet_name=f"Precios Oferta {datetime.today().strftime('%Y-%m-%d')}", index=False)  # tabla 12
    df_precios_stock_mediano.to_excel(writer, sheet_name=f"Precios Stock Mediano Ayer", index=False)  # tabla 17
    opt.to_excel(writer, sheet_name=f"OPT - Media Precios Compet", index=False)  # tabla 13
    e_7.to_excel(writer, sheet_name=f"Numero de Locales Activos Ayer", index=False)  # tabla 14
    e_8.to_excel(writer, sheet_name=f"Days on Hand - Articulos", index=False)  # tabla 15
    e_9.to_excel(writer, sheet_name=f"Days on Hand - Subclases", index=False)  # tabla 16


In [ ]:
sys.exit()

In [ ]:
with pd.ExcelWriter(f"C:\\Users\\leonardo.mangold\\PycharmProjects\\analisis\\kpis_promos_mauri\\Entregas\\Promos {datetime.today().strftime('%Y-%m-%d')}.xlsx", engine='xlsxwriter') as writer:
    e_1.to_excel(writer, sheet_name='Articulos Publicados', index=False) # tabla 1
    arts_2.to_excel(writer, sheet_name='Venta y GB1 por Promo', index=False) # tabla 2
    e_6.to_excel(writer, sheet_name='Numero de Subclases', index=False) # tabla 3
    e_2.to_excel(writer, sheet_name='Venta y GB1 por Subclase', index=False) # tabla 4
    e_3.to_excel(writer, sheet_name='Venta y GB1 Promo Agrupado', index=False) # tabla 5
    e_4.to_excel(writer, sheet_name='Num Articulos por Subclase', index=False) # tabla 6
    ace.to_excel(writer, sheet_name='Aceleracion', index=False) # tabla 7
    ace_2.to_excel(writer, sheet_name='Aceleracion por Subclase', index=False) # tabla 8
    ace_3.to_excel(writer, sheet_name='Aceleracion Resumen', index=False) # tabla 9
    e_5.to_excel(writer, sheet_name='Tickets', index=False) # tabla 10
    eventos.to_excel(writer, sheet_name='Numero de Eventos', index=False) # tabla 11
    precios.to_excel(writer, sheet_name=f"Precios {datetime.today().strftime('%Y-%m-%d')}", index=False) # tabla 12
    opt.to_excel(writer, sheet_name=f"OPT - Media Precios Compet", index=False) # tabla 13
    e_7.to_excel(writer, sheet_name=f"Numero de Locales Activos Ayer", index=False) # tabla 14
    e_8.to_excel(writer, sheet_name=f"Days on Hand - Articulos", index=False) # tabla 15
    e_9.to_excel(writer, sheet_name=f"Days on Hand - Subclases", index=False) # tabla 16


In [14]:
datetime.today().strftime('%Y-%m-%d')

'2023-12-13'

# Fin